# GRPC Inference

### Setup

In [ ]:
!pip install grpcio grpcio-tools

In [ ]:
!pip install opencv-python-headless

In [ ]:
grpc_host = 'modelmesh-serving'
grpc_port = 8033
model_name = 'rps'
input_name = 'input'

In [ ]:
import grpc
import grpc_predict_v2_pb2
import grpc_predict_v2_pb2_grpc


channel = grpc.insecure_channel(f"{grpc_host}:{grpc_port}")
stub = grpc_predict_v2_pb2_grpc.GRPCInferenceServiceStub(channel)

request = grpc_predict_v2_pb2.ModelMetadataRequest(name=model_name)
response = stub.ModelMetadata(request)
response

### Preprocess image

Preprocessing taken from:

https://pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html

https://github.com/openvinotoolkit/model_server/blob/main/demos/using_onnx_model/python/onnx_model_demo.py

In [ ]:
import sys
import argparse
import numpy as np
import cv2

def preprocess(img_data):
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):
        # for each pixel in each channel, divide the value by 255 to get value between [0, 1] and then normalize
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
    return norm_img_data

def getJpeg(path, size):
    with open(path, mode='rb') as file:
        content = file.read()

    img = np.frombuffer(content, dtype=np.uint8)
    img = cv2.imdecode(img, cv2.IMREAD_COLOR)  # BGR format
    # format of data is HWC
    # add image preprocessing if needed by the model
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32')
    #convert to NCHW
    img = img.transpose(2,0,1)
    # normalize to adjust to model training dataset
    img = preprocess(img)
    img = img.reshape(1,3,size,size)
    # print(path, img.shape, "; data range:",np.amin(img),":",np.amax(img))
    return img



### Request

In [ ]:
import time


def infer(img, size=224, debug=False):
    # request content building
    inputs = []
    inputs.append(grpc_predict_v2_pb2.ModelInferRequest().InferInputTensor())
    inputs[0].name = input_name
    inputs[0].datatype = "FP32"
    inputs[0].shape.extend([1, 3, size, size])
    arr = img.flatten()
    inputs[0].contents.fp32_contents.extend(arr)

    # request building
    request = grpc_predict_v2_pb2.ModelInferRequest()
    request.model_name = model_name
    request.inputs.extend(inputs)

    t1 = time.time()
    response = stub.ModelInfer(request)
    t2 = time.time()
    inference_time = t2-t1
    arr = np.frombuffer(response.raw_output_contents[0], dtype=np.float32)
    max = np.argmax(arr)
    class_name = classes.rps_classes[max]
    
    if debug:        
        print(f"Request took {inference_time}")
        print(arr)
        print("Class is with highest score: {}".format(max))
        print("Detected class name: {}".format(class_name))
        
    return class_name

In [ ]:
import classes
import os

for root,dirs,files in os.walk('images'):
    for file in files:
        image_path = os.path.join(root, file)
        print(f'Classifying {image_path}')
        jpeg = getJpeg(image_path, 224)
        class_name = infer(jpeg, 224, False)
        print("Detected class name: {}".format(class_name))

